# Python Lambda Calculus

## Booleans

In [1]:
TRUE = lambda x: lambda y: x

In [2]:
TRUE.__name__ = "True"

In [3]:
print(TRUE)

<function <lambda> at 0x0000017AFD3B8598>


In [4]:
FALSE = lambda x: lambda y: y

In [5]:
FALSE.__name__ = "False"

### Utility Functions

In [6]:
toBool = lambda x: x(True)(False)

In [7]:
toBool(TRUE)

True

In [8]:
toBool(FALSE)

False

### Logical Operators

In [9]:
AND = lambda x: lambda y: x(y)(FALSE)

In [10]:
OR = lambda x: lambda y: x(TRUE)(y)

In [11]:
NOT = lambda x: x(FALSE)(TRUE)

In [12]:
XOR = lambda x:  lambda y: x(NOT(y))(y)

## Natural Numbers

In [13]:
ZERO = lambda f: lambda x: x

In [14]:
ONE = lambda f: lambda x: f(x)

In [15]:
TWO = lambda f: lambda x: f(f(x))

In [16]:
THREE = lambda f: lambda x: f(f(f(x)))

In [17]:
FOUR = lambda f: lambda x: f(f(f(f(x))))

### Number Functions

In [18]:
SUCC = lambda n: lambda f: lambda x: f(n(f)(x))

In [19]:
PAIR = lambda a: lambda b: lambda f: f(a)(b)

Basically PAIR needs a function to work with either a or b, that function is f

FST takes in a PAIR, and then sends in a function to it, here we send the TRUE function, since it will give me the first argument

In [20]:
FST = lambda p: p(TRUE)

In [21]:
vim = PAIR(TRUE)(FALSE)

In [22]:
FST(vim)('T')('F')

'T'

In [23]:
SND = lambda p: p(FALSE)

In [24]:
SND(vim)('T')('F')

'F'

PHI takes in a pair and then returns a pair in which the first element is the second element of the argument and the second element is the successor of the second argument, basically the first argument is useless, for a single call of this function.

copy 2nd to 1st, and increment 2nd

In [25]:
PHI = lambda p: PAIR(SND(p))(SUCC(SND(p)))

In [26]:
FST(PHI(PAIR(ZERO)(ZERO)))(lambda x: x + 1)(0)

0

In [27]:
SND(PHI(PAIR(ZERO)(ZERO)))(lambda x: x + 1)(0)

1

Guess what we are doing !, if we apply PHI n times to a pair  
```ZERO PHI(ZERO, ZERO) = (ZERO, ZERO)
ONE  PHI(ZERO, ZERO) = (ZERO, ONE)
TWO  PHI(ZERO, ZERO) = (ONE , TWO)```   
and so on

Voila ! we've got our predecessor working, in a really easy way

In [28]:
FST(FOUR(PHI)(PAIR(ZERO)(ZERO)))(lambda x: x + 1)(0)

3

I hate this form of PRED, dammit

In [29]:
PRED = lambda n: n(lambda p: lambda z: z(SUCC(p(TRUE)))(p(TRUE)))(lambda z: z(ZERO)(ZERO))(FALSE)

aaaahhhhhhhhhh much better

In [30]:
PRED = lambda n: FST(n(PHI)(PAIR(ZERO)(ZERO)))

In [31]:
PLUS = lambda k: lambda n: k(SUCC)(n)

In [32]:
MULT = lambda k: lambda n: lambda f: n(k(f))

In [33]:
MULT = lambda k: lambda n: k(PLUS(n))(ZERO)

In [34]:
EXP = lambda k: lambda n: n(k)

In [35]:
MINUS = lambda k: lambda n: n(PRED)(k)

The way IS_ZERO works is that ZERO is basically nothing applied to x, its just x, if the number passed to IS_ZERO was ZERO, then it would apply f to x, zero number of times, the function passed is useless, so the second argument is returned which is TRUE, but if its ONE, TWO, THREE they will apply the function to 1, 2, 3 times respectively, so the function should always return FALSE, you know TRUE(FALSE) is always false, TRUE will always return the first argument which is FALSE, and Ta-Da, you got the IS_ZERO function

In [36]:
IS_ZERO = lambda n: n(TRUE(FALSE))(TRUE)

### Utility Functions

In [37]:
toInt = lambda x: x(lambda n: n + 1)(0)

In [38]:
toInt(FOUR)

4

In [39]:
toInt(SUCC(FOUR))

5

In [40]:
toInt(PRED(FOUR))

3

In [41]:
toInt(PLUS(FOUR)(THREE))

7

In [42]:
toInt(MULT(FOUR)(THREE))

12

In [43]:
toInt(EXP(TWO)(FOUR))

16

In [44]:
toInt(MINUS(FOUR)(TWO))

2

In [45]:
toBool(IS_ZERO(ZERO))

True

In [46]:
toBool(IS_ZERO(FOUR))

False

## Logical Operators

In [47]:
LESS_THAN_OR_EQUAL = lambda n: lambda m: IS_ZERO(MINUS(n)(m))

In [48]:
LESS_THAN = lambda n: lambda m: AND(LESS_THAN_OR_EQUAL(n)(m)) \
                                   (NOT(IS_ZERO(n(PRED)(m))))

In [49]:
EQUALS = lambda n: lambda m: AND(LESS_THAN_OR_EQUAL(n)(m))(LESS_THAN_OR_EQUAL(m)(n))

In [50]:
GREATER_THAN = lambda n: lambda m: NOT(LESS_THAN_OR_EQUAL(n)(m))

In [51]:
toInt(MINUS(ZERO)(FOUR))

0

In [52]:
toBool(LESS_THAN_OR_EQUAL(TWO)(THREE))

True

In [53]:
toBool(LESS_THAN_OR_EQUAL(FOUR)(ZERO))

False

In [54]:
toBool(LESS_THAN(THREE)(FOUR))

True

In [55]:
toBool(LESS_THAN(THREE)(THREE))

False

In [56]:
toBool(EQUALS(TWO)(TWO))

True

In [57]:
toBool(EQUALS(THREE)(TWO))

False

In [58]:
toBool(GREATER_THAN(THREE)(TWO))

True

In [59]:
toBool(GREATER_THAN(TWO)(THREE))

False

In [60]:
COMPOSE = lambda f: lambda g: lambda x: f(g(x))

## The Fixpoint "Z" Combinator

In [61]:
FIX = lambda f: (lambda x: f(lambda y: x(x)(y)))(lambda x: f(lambda y: x(x)(y)))

In [62]:
MOD = FIX(lambda Y: lambda n: lambda m: \
        LESS_THAN_OR_EQUAL(m)(n) \
        (lambda x: Y(MINUS(n)(m))(m)(x)) \
        (n))

In [63]:
DIV = FIX(lambda Y: lambda n: lambda m: \
        LESS_THAN_OR_EQUAL(m)(n) \
        (lambda x: SUCC(Y(MINUS(n)(m))(m))(x)) \
        (ZERO))

In [64]:
toInt(MOD(THREE)(TWO))

1

In [65]:
toInt(DIV(FOUR)(TWO))

2

### Lists

In [66]:
LIST_ELEMENT = lambda x: lambda xs: PAIR(FALSE)(PAIR(x)(xs))

In [67]:
EMPTY_LIST = PAIR(TRUE)(TRUE)

In [68]:
IS_EMPTY = FST

In [69]:
HEAD = lambda xs: FST(SND(xs))

In [70]:
TAIL = lambda xs: SND(SND(xs))

## Factorial

In [71]:
FACT = lambda r: lambda n: IS_ZERO(n)(ONE) \
                                     (lambda x: MULT(n)(r(PRED(n)))(x))

In [72]:
FACT = FIX(FACT)

In [73]:
TEN = MULT(TWO)(PLUS(FOUR)(ONE))

In [74]:
FIVE = PLUS(FOUR)(ONE)

In [75]:
toInt(FACT(FIVE))

120

## Fibonacci

In [79]:
FIB = lambda fib: lambda n: IS_ZERO(n)(ZERO)( \
                           EQUALS(n)(ONE)(ONE) \
                           (lambda x: PLUS(fib(PRED(n)))(fib(PRED(PRED(n))))(x)))

In [80]:
FIB = FIX(FIB)

In [81]:
SEVEN = PLUS(FOUR)(THREE)

In [83]:
toInt(FIB(SEVEN))

13

# Unit Testing

In [28]:
import unittest

In [29]:
class Testing(unittest.TestCase):
    def test_booleans(self):
        self.assertEqual(AND(TRUE)(TRUE), TRUE)
        self.assertEqual(OR(FALSE)(FALSE), FALSE)
        self.assertEqual(NOT(FALSE), TRUE)
        self.assertEqual(XOR(TRUE)(TRUE), FALSE)
    def test_naturals(self):
        self.assertEqual(toInt(ZERO), 0)
        self.assertEqual(toInt(ONE), 1)
        self.assertEqual(toInt(TWO), 2)
        self.assertEqual(toInt(THREE), 3)
        self.assertEqual(toInt(FOUR), 4)
        self.assertEqual(toInt(SUCC(FOUR)), 5)
        self.assertEqual(toInt(PLUS(FOUR)(THREE)), 7)

In [30]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.013s

OK


In [84]:
[(lambda x: '👧🐶' if x%15==0 else '🐶' if x%3==0 else '👧' if x%5==0 else x )(i) for i in range(100)]

['👧🐶',
 1,
 2,
 '🐶',
 4,
 '👧',
 '🐶',
 7,
 8,
 '🐶',
 '👧',
 11,
 '🐶',
 13,
 14,
 '👧🐶',
 16,
 17,
 '🐶',
 19,
 '👧',
 '🐶',
 22,
 23,
 '🐶',
 '👧',
 26,
 '🐶',
 28,
 29,
 '👧🐶',
 31,
 32,
 '🐶',
 34,
 '👧',
 '🐶',
 37,
 38,
 '🐶',
 '👧',
 41,
 '🐶',
 43,
 44,
 '👧🐶',
 46,
 47,
 '🐶',
 49,
 '👧',
 '🐶',
 52,
 53,
 '🐶',
 '👧',
 56,
 '🐶',
 58,
 59,
 '👧🐶',
 61,
 62,
 '🐶',
 64,
 '👧',
 '🐶',
 67,
 68,
 '🐶',
 '👧',
 71,
 '🐶',
 73,
 74,
 '👧🐶',
 76,
 77,
 '🐶',
 79,
 '👧',
 '🐶',
 82,
 83,
 '🐶',
 '👧',
 86,
 '🐶',
 88,
 89,
 '👧🐶',
 91,
 92,
 '🐶',
 94,
 '👧',
 '🐶',
 97,
 98,
 '🐶']